# Developing Brain Atlas through Deep Learning 
## A. Iqbal, R. Khan, T. Karayannis
### Source : https://github.com/itsasimiqbal/SeBRe

## Modified by 

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib

import matplotlib.pyplot as plt
import pandas as pd
from config import Config
import utils
import model as modellib
import visualize
from model import log
from PIL import Image

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()
pd.set_option('display.max_rows', 5000)
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
os.chdir('/home/hamza/Allen_brain/test/')
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

/home/hamza/virtualENV/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hamza/virtualENV/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hamza/virtualENV/python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hamza/virtualENV/python36/lib/python3.6/site-pack

In [2]:
import glob #for selecting png files in training images folder
from natsort import natsorted, ns #for sorting filenames in a directory
import skimage
from skimage import io

In [6]:
class BrainConfig(Config):
    """Configuration for training on the brain dataset.
    Derives from the base Config class and overrides values specific
    to the brain dataset.
    """
    # Give the configuration a recognizable name
    NAME = "brain"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 #8 ; reduced to avoid running out of memory when image size increased

    # Number of classes (including background)
    NUM_CLASSES = 1 + 324  # background + 8 regions

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 512*3 #128
    IMAGE_MAX_DIM = 2048*3#128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 2000 #100 #steps_per_epoch: Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. 
                          #steps_per_epoch = TotalTrainingSamples / TrainingBatchSize (default to use entire training data per epoch; can modify if required)
                          
    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 100 #5 #validation_steps = TotalvalidationSamples / ValidationBatchSize
                         #Ideally, you use all your validation data at once. If you use only part of your validation data, you will get different metrics for each batch, 
                         #what may make you think that your model got worse or better when it actually didn't, you just measured different validation sets.
                         #That's why they suggest validation_steps = uniqueValidationData / batchSize. 
                         #Theoretically, you test your entire data every epoch, as you theoretically should also train your entire data every epoch.
                         #https://stackoverflow.com/questions/45943675/meaning-of-validation-steps-in-keras-sequential-fit-generator-parameter-list
    

    
    ###### Further changes (experimentation):
    
     # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 61 #100 #decreased to avoid duplicate instances of each brain region
    
    # Max number of final detections
    DETECTION_MAX_INSTANCES = 61 #100 # #decreased to avoid duplicate instances of each brain region

    # Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE =  0.9 #0.7

    # Non-maximum suppression threshold for detection
    DETECTION_NMS_THRESHOLD = 0.3 # if overlap ratio is greater than the overlap threshold (0.3), suppress object (https://www.pyimagesearch.com/2014/11/17/non-maximum-suppression-object-detection-python)

        
    
    
config = BrainConfig()
config.display()


Configurations:
BACKBONE_SHAPES                [[1536 1536]
 [ 768  768]
 [ 384  384]
 [ 192  192]
 [  96   96]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        61
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  6144
IMAGE_MIN_DIM                  1536
IMAGE_PADDING                  True
IMAGE_SHAPE                    [6144 6144    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               61
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           brain
NUM_CLASSES                    325
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TR

In [7]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset

Load training dataset

Extend the Dataset class and add a method to load the brain sections dataset, `load_brain()`, and override the following methods:

* load_image()
* load_mask()
* image_reference() # do not need to for now

In [8]:
all_structues = (pd
                 .read_csv("complet.csv")
                 .pipe(lambda x: x.loc[x.depth==7,['atlas_id','name']])
                 .assign(name = lambda x: [ re.sub(r'\W+','',re.sub(r'\s+','_',str(i))) for i in x.name]))

 


all_structues.head()

,atlas_id,name
7,68,Frontal_pole_layer_1
8,667,Frontal_pole_layer_23
9,526157192,Frontal_pole_layer_5
10,526157196,Frontal_pole_layer_6a
11,526322264,Frontal_pole_layer_6b


In [9]:
########### Create training dataset:

class BrainDataset_Train(utils.Dataset):
    """Generates the brain section dataset. The dataset consists of locally stored 
    brain section images, to which file access is required.
    """

    #see utils.py for default def load_image() function; modify according to your dataset
    
    def load_brain(self): 
        """
        for naming image files follow this convention: '*_(image_id).jpg'
        """
        
        os.chdir('/home/hamza/Allen_brain/test/')
        for idx, i in enumerate(all_structues.name.tolist()):
            self.add_class('brain',str(idx),i)
    

        
        
        training_images_folder = 'Train/Training_JPG'
        os.chdir(training_images_folder)
        
        cwd = os.getcwd()
        img_list = glob.glob('*.jpg')
        img_list = natsorted(img_list, key=lambda y: y.lower())
        for i in img_list:  #image_ids start at 0 (to keep correspondence with load_mask which begins at image_id=0)!
            img = skimage.io.imread(i) #grayscale = 0
            im_dims = np.shape(img)
            self.add_image("brain", image_id=int(i[:-4]), path = cwd+'/'+i,height = im_dims[0], width = im_dims[1])#, depth = im_dims[2])
            
            #print(im_dims)
      
            
    
    def load_mask(self,image_id):
        """Load instance masks for the given image.
        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].

        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks."""
        
        os.chdir('/home/hamza/Allen_brain/test/')
        print(image_id)
        masks_folder = 'Train/Training_Mask'
        os.chdir(masks_folder)
        subfolder = str(self.image_info[image_id]['id'])#add 1 to image_id, to get to correct corresponding masks folder for a given image 
        print(subfolder)
        os.chdir(subfolder) 
        
        info = info = self.image_info[image_id]    
        
        
        
        mk_list = glob.glob('*.png')
        
        count = len(mk_list)
        mk_id = 0
        mask = np.zeros([info['height'], info['width'], count], dtype=np.uint8)        
        class_ids = np.zeros(count)
        
        for m in mk_list:
            bin_mask = np.array(Image.open(m).convert('1')) # grayscale=0
            mk_size = np.shape(bin_mask)
            mask[:, :, mk_id]= bin_mask
            
            # Map class names to class IDs.
            print(m)
            class_ids[mk_id] = all_structues.loc[all_structues.name==m[:-9],'atlas_id'] #fifth last position from mask_image name = class_id #need to update(range) if class_ids become two/three-digit numbers 
            mk_id += 1
        return mask, class_ids.astype(np.int32)
    
    
    

########### Create validation dataset:   

class BrainDataset_Val(utils.Dataset):
    """Generates the brain section dataset. The dataset consists of locally stored 
    brain section images, to which file access is required.
    """

    #see utils.py for default def load_image() function; modify according to your dataset
    
    def load_brain(self): 
        """
        for naming image files follow this convention: '*_(image_id+1).jpg'
        """
        
        os.chdir('/home/hamza/Allen_brain/test/')
        for idx, i in enumerate(all_structues.name.tolist()):
            self.add_class('brain',str(idx),i)
        
        
        
        training_images_folder = 'Val/Val_JPG'
        os.chdir(training_images_folder)
        
        cwd = os.getcwd()
        img_list = glob.glob('*.jpg')
        img_list = natsorted(img_list, key=lambda y: y.lower())
        for i in img_list:  #image_ids start at 0 (to keep correspondence with load_mask which begins at image_id=0)!
            img = skimage.io.imread(i) #grayscale = 0
            im_dims = np.shape(img)
            self.add_image("brain", image_id=int(i[:-4]), path = cwd+'/'+i,height = im_dims[0], width = im_dims[1])#, depth = im_dims[2])
            
            #print(im_dims)
      
 
    def load_mask(self,image_id):
        """Load instance masks for the given image.
        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].

        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks."""
        
        os.chdir('/home/hamza/Allen_brain/test/')
        print(image_id)
        masks_folder = 'Val/Val_Mask'
        os.chdir(masks_folder)
        subfolder = str(self.image_info[image_id]['id'])#add 1 to image_id, to get to correct corresponding masks folder for a given image 
        print(subfolder)
        os.chdir(subfolder) 
        
        info = self.image_info[image_id]   
        
        
        
        mk_list = glob.glob('*.png')
        
        count = len(mk_list)
        mk_id = 0
        mask = np.zeros([info['height'], info['width'], count], dtype=np.uint8)        
        class_ids = np.zeros(count)
        
        for m in mk_list:
            bin_mask = np.array(Image.open(m).convert('1')) # grayscale=0
            mk_size = np.shape(bin_mask)
            mask[:, :, mk_id]= bin_mask
            
            # Map class names to class IDs.
            print(m)
            class_ids[mk_id] = all_structues.loc[all_structues.name==m[:-9],'atlas_id'] #fifth last position from mask_image name = class_id #need to update(range) if class_ids become two/three-digit numbers 
            mk_id += 1
        return mask, class_ids.astype(np.int32)
    
    
    


In [10]:
# Training dataset
dataset_train = BrainDataset_Train()
dataset_train.load_brain()
dataset_train.prepare() #does nothing for now 

dataset_val = BrainDataset_Val()
dataset_val.load_brain()
dataset_val.prepare()#does nothing for now 

In [11]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=2, 
            layers='heads') #epochs = 1


Starting at epoch 0. LR=0.001

Checkpoint Path: /home/hamza/Allen_brain/test/logs/brain20200120T1315/mask_rcnn_brain_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn

/home/hamza/virtualENV/python36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/hamza/virtualENV/python36/lib/python3.6/site-packages/keras/engine/training.py:2039: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/2
5
100960496
2
100960484
Main_olfactory_bulb_mitral_layer_mask.png
Main_olfactory_bulb_mitral_layer_mask.png
Main_olfactory_bulb_outer_plexiform_layer_mask.png
Main_olfactory_bulb_outer_plexiform_layer_mask.png
Main_olfactory_bulb_glomerular_layer_mask.png
Main_olfactory_bulb_glomerular_layer_mask.png
Main_olfactory_bulb_granule_layer_mask.png
Main_olfactory_bulb_granule_layer_mask.png
Main_olfactory_bulb_inner_plexiform_layer_mask.png
original image shape:  (4416, 4608, 3)
Main_olfactory_bulb_inner_plexiform_layer_mask.png
original image shape:  (4352, 4672, 3)
6
100960500
7
100960504
Main_olfactory_bulb_mitral_layer_mask.png
Main_olfactory_bulb_mitral_layer_mask.png
Main_olfactory_bulb_outer_plexiform_layer_mask.png
Main_olfactory_bulb_outer_plexiform_layer_mask.png
Main_olfactory_bulb_glomerular_layer_mask.png
Main_olfactory_bulb_glomerular_layer_mask.png
Main_olfactory_bulb_granule_layer_mask.png
Main_olfactory_bulb_granule_layer_mask.png
Main_olfactory_bulb_inner_plexifor

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=3, 
            layers="all")#layers="heads" ; epochs = 2

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)